First, import numpy and scipy library.

In [14]:
import numpy as np
import scipy.optimize as sopt

Define the cost function $t(x)$ abd its integration function, which is the objective function of SO. 

In [15]:
def t(x):
    x1, x2, x3 = x
    t1 = 10*(1+0.15*(x1/2)**4)
    t2 = 20*(1+0.15*(x2/4)**4)
    t3 = 25*(1+0.15*(x3/3)**4)
    return [t1, t2, t3]

def sum_tilde_t(x):
    x1, x2, x3 = x
    tilde_t1 = 10*(1+0.15*(x1/2)**4)*x1
    tilde_t2 = 20*(1+0.15*(x2/4)**4)*x2
    tilde_t3 = 25*(1+0.15*(x3/3)**4)*x3
    return tilde_t1 + tilde_t2 + tilde_t3

Initialize the set of points as the list named guesses. Define the total flow in three routes. 

In [16]:
guesses = []
tot_flow = 10.0

Function $step$: contain task for one iteration.
1. Define and calulate cost $t(x)$ and objective function $sum\_tilde\_t(x)$.
2. Find drection $y$: Set the flow direction $y$ to the route with minimum cost. 
3. Find the step size $\alpha \in (0,1)$ 
    1. Define the objective function after moving as $func$.
    2. Define the constraints $con$ to satisfy Total flow conservation and bound $\alpha \in (0,1)$.
    3. Use $scipy.optimize.minimize$ tool to obtain the value of step size $\alpha$.
5. Update flow as $x^{n+1} = x^n + \alpha * (y^n-x^n)$
6. Do convergence test. If satisfy the convergence test, stop the iteration; if not, enter next iteration. 


In [17]:
def step(i):
    print('in step',i+1)
    
    x = guesses[-1]

    t_array = t(x)
    Z = sum_tilde_t(x)
    
    print('\tUpdate \t\tt\t', np.round(t_array,decimals=3), '\tZ(x)\t', np.round(Z,decimals=2))
    
    # find y with min t
    y_index = np.argmin(t_array)
    y = np.array([0.0, 0.0, 0.0])
    y[y_index] = tot_flow

    print('\tDirection \ty\t', np.round(y,decimals=2))

    # objective function to minimize
    def func(alpha):
        return sum_tilde_t(x+alpha*(y-x))

    # constrain: sum of flow equals to total flow
    def con(alpha):
        return np.sum(x+alpha*(y-x)) - tot_flow
    
    cons = {'type':'eq', 'fun': con}
    
    # find alpha that minimize objective function
    alpha = np.array([0.0, 0.0, 0.0])
    bounds = [(0,1) for i in range(3)]
    res = sopt.minimize(func, alpha, bounds=bounds, constraints=cons)
    alpha = res.x
    print('\tStep \t\talpha\t', np.round(alpha,decimals=3))

    # update x
    next_guess = x + alpha * (y-x)
    guesses.append(next_guess)
    print('\tMove \t\tx\t', np.round(next_guess,decimals=3))
    

In [18]:
def Converge_test(guesses):
    delta = 0.01
    x_01, x_11, x_21 = guesses[-2]
    x_02, x_12, x_22 = guesses[-1]
    temp = np.sqrt((x_02-x_01)**2 + (x_11-x_12)**2 + (x_21-x_22)**2)/(x_01+x_11+x_12)
    return True if (temp < delta) else False

main function to solve the Frank Wolfe for SO problem. 

In [19]:
if __name__ == '__main__':
    guesses = [np.array([10.0, 0.0, 0.0])]
    print('Init Guesses\t\tx\t',guesses[0])

    for i in range(3):
        step(i)
        if Converge_test == True:
            break
    

Init Guesses		x	 [10.  0.  0.]
in step 1
	Update 		t	 [947.5  20.   25. ] 	Z(x)	 9475.0
	Direction 	y	 [ 0. 10.  0.]
	Step 		alpha	 [0.621 0.621 0.   ]
	Move 		x	 [3.793 6.207 0.   ]
in step 2
	Update 		t	 [29.398 37.398 25.   ] 	Z(x)	 343.64
	Direction 	y	 [ 0.  0. 10.]
	Step 		alpha	 [0.252 0.305 0.285]
	Move 		x	 [2.835 4.314 2.851]
in step 3
	Update 		t	 [16.058 24.058 28.058] 	Z(x)	 229.3
	Direction 	y	 [10.  0.  0.]
	Step 		alpha	 [0. 0. 0.]
	Move 		x	 [2.835 4.314 2.851]
